# Segmented Regression

In [33]:
import gc
import os
from typing import Dict, List, Tuple

import dask.bag as db
import numba
import numpy as np
import pandas as pd
import seaborn as sns
from dask.diagnostics import ProgressBar
from matplotlib import pyplot as plt
from scipy import stats
from tqdm.notebook import tqdm

## Read Trade Data

In [2]:
# Trades are sorted by trade_id already(except BitMEX, which is sorted by timestamp)
PER_TRADE_DATA_DIR = '/data/csv'

In [3]:
BTC_PAIRS = [
    ('Binance', 'Spot', 'BTC_USDT'),
    ('Binance', 'Swap', 'BTC_USDT'),
    ('BitMEX', 'Swap', 'BTC_USD'),
    ('Huobi', 'Spot', 'BTC_USDT'),
    ('Huobi', 'Swap', 'BTC_USD'),
    ('OKEx', 'Spot', 'BTC_USDT'),
    ('OKEx', 'Swap', 'BTC_USDT'),
    ('OKEx', 'Swap', 'BTC_USD'),
]

ETH_PAIRS = [
    ('Binance', 'Spot', 'ETH_USDT'),
    ('Binance', 'Swap', 'ETH_USDT'),
    ('BitMEX', 'Swap', 'ETH_USD'),
    ('Huobi', 'Spot', 'ETH_USDT'),
    ('Huobi', 'Swap', 'ETH_USD'),
    ('OKEx', 'Spot', 'ETH_USDT'),
    ('OKEx', 'Swap', 'ETH_USDT'),
    ('OKEx', 'Swap', 'ETH_USD'),
]

In [4]:
def get_csv_file(exchange: str, market_type: str, pair: str)->str:
    assert market_type == 'Spot' or market_type == 'Swap'
    return os.path.join(PER_TRADE_DATA_DIR, f'{exchange}.{market_type}.{pair}.csv')

In [5]:
get_csv_file(*BTC_PAIRS[0])

'/data/csv/Binance.Spot.BTC_USDT.csv'

In [6]:
get_csv_file(*ETH_PAIRS[-1])

'/data/csv/OKEx.Swap.ETH_USD.csv'

In [7]:
def read_csv(trade_csv_file: str)->pd.DataFrame:
    df = pd.read_csv(trade_csv_file, engine='c',
                     dtype={'exchange': 'category', 'marketType': 'category', 'pair': 'category',
                            'timestamp': 'int64', 'price': 'float64',
                            'quantity': 'float64', 'side': 'bool', 'trade_id': 'string'},
                     usecols=['timestamp', 'price', 'quantity'])
    return df

In [8]:
okex_swap_eth_usd = read_csv(get_csv_file(*ETH_PAIRS[-1]))

In [9]:
okex_swap_eth_usd.head()

,timestamp,price,quantity
0,1588291218207,206.18,3.880105
1,1588291218207,206.18,2.134058
2,1588291218207,206.18,2.425065
3,1588291218207,206.15,2.279893
4,1588291218207,206.15,1.261218


## Ordinary Least Square

In [10]:
# see https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html
#@numba.njit()
def ols(X: np.ndarray, y: np.ndarray)->np.ndarray:
    if np.unique(X).size < 2:
        return y
    y_first = y[0]
    X = X - X[0]
    y = y - y[0]
    A = np.vstack((X, np.ones(len(X)))).T
    m, b = np.linalg.lstsq(A, y, rcond=None)[0]
    y_hat = m * X + b + y_first
    return y_hat

In [11]:
ols(np.array([0, 1, 2, 3]), np.array([-1, 0.2, 0.9, 2.1]))

array([-0.95,  0.05,  1.05,  2.05])

In [12]:
# see https://devarea.com/linear-regression-with-numpy/
@numba.njit(fastmath=True, parallel=True)
def ols_1d(X: np.ndarray, y: np.ndarray)->np.ndarray:
    if np.unique(X).size < 2:
        return y
    y_first = y[0]
    X = X - X[0]
    y = y - y[0]
    m = (len(X) * np.sum(X*y) - np.sum(X) * np.sum(y)) / (len(X)*np.sum(X*X) - np.sum(X) * np.sum(X))
    b = (np.sum(y) - m *np.sum(X)) / len(X)
    y_hat = m * X + b + y_first
    return y_hat

In [13]:
ols_1d(np.array([0, 1, 2, 3]), np.array([-1, 0.2, 0.9, 2.1]))

array([-0.95,  0.05,  1.05,  2.05])

In [14]:
# from https://machinelearningmastery.com/probabilistic-model-selection-measures/

# calculate aic for regression
def calculate_aic(n, mse, num_params):
    aic = n * np.log(mse) + 2 * num_params
    return aic

# calculate bic for regression
def calculate_bic(n, mse, num_params):
    bic = n * np.log(mse) + num_params * np.log(n)
    return bic

In [15]:
def calc_stats(Y: np.ndarray, Y_hat: np.ndarray)->Dict:
    assert Y.shape == Y_hat.shape
    n = Y.shape[0]
    squared_error = np.sum(np.power(Y - Y_hat, 2))
    variance = np.sum(np.power(Y- np.mean(Y), 2))
    r_square = (variance-squared_error)/variance
    mse = squared_error / n
    mae = np.sum(np.abs(Y - Y_hat)) / n
    return {
        'R2': r_square,
        'MSE': mse,
        'MAE': mae,
        'AIC': calculate_aic(n, mse, 1),
        'BIC': calculate_bic(n, mse, 1),
    }

## Segmented Linear Regression

In [16]:
def segmented_linear_regression(csv_file: str, bar_type: str, bar_size)->Dict:
    df = read_csv(csv_file)
    if bar_type == 'TimeBar':
        df['bar_index'] = df['timestamp'] // bar_size
    elif bar_type == 'TickBar':
        df['bar_index'] = (df.index // bar_size).to_series().reset_index(drop=True)
    elif bar_type == 'VolumeBar':
        df['bar_index'] = df['quantity'].astype('float64').cumsum().floordiv(bar_size).astype('uint32')
    elif bar_type == 'DollarBar':
        df['bar_index'] = (df['quantity'] * df['price']).astype('float64').cumsum().floordiv(bar_size).astype('uint32')

    df = df[['bar_index','timestamp', 'price']]  # remove quantity column

    grouped = df.groupby('bar_index').agg(list)
    grouped['timestamp'] = grouped['timestamp'].apply(np.array)
    grouped['price'] = grouped['price'].apply(np.array)

    predicted = grouped.apply(lambda row: ols_1d(row['timestamp'], row['price']), axis=1)
    Y_hat = np.concatenate(predicted.values)
    stats = calc_stats(df['price'].values, Y_hat)

    del Y_hat
    del predicted
    del grouped
    del df
    gc.collect()
    
    exchange, market_type, pair, _ = os.path.basename(csv_file).split('.')
    result = {
        'exchange': exchange,
        'market_type': market_type,
        'pair': pair,
        'bar_type': bar_type,
        'bar_size': bar_size,
    }
    result.update(stats)
    return result

In [17]:
segmented_linear_regression(get_csv_file(*ETH_PAIRS[-1]), 'TimeBar', 60000)

{'exchange': 'OKEx',
 'market_type': 'Swap',
 'pair': 'ETH_USD',
 'bar_type': 'TimeBar',
 'bar_size': 60000,
 'R2': 0.9998944530775309,
 'MSE': 0.0309951742931635,
 'MAE': 0.0885760890407249,
 'AIC': -10769700.097370578,
 'BIC': -10769687.15040791}

## Compare different bars

In [18]:
TIME_BAR_SIZES = {
  'BTC': [4000, 8000, 10000],
  'ETH': [4000, 8000, 10000],
}

TICK_BAR_SIZES = {
  'BTC': [16, 32, 64, 128],
  'ETH': [8, 16, 32, 64],
}

VOLUME_BAR_SIZES = {
  'BTC': [1, 2, 4, 8, 16, 32],
  'ETH': [16, 32, 64, 128, 256, 512],
}

DOLLAR_BAR_SIZES = {
  'BTC': [10000, 20000, 40000, 80000, 160000, 320000],
  'ETH': [4000, 8000, 16000, 32000, 64000, 128000],
}

In [19]:
def gen_tasks(exchange_market_pairs: List[Tuple[str, str, str]], bar_type: str, bar_sizes: List[int])->None:
    csv_files = [get_csv_file(*exchange_market_pair) for exchange_market_pair in exchange_market_pairs]
    tasks = [(csv_file, bar_type, bar_size) for csv_file in csv_files for bar_size in bar_sizes]
    return tasks

In [20]:
def batch(base: str)->pd.DataFrame:
    exchange_market_pairs = BTC_PAIRS if base == 'BTC' else ETH_PAIRS
    tasks = gen_tasks(exchange_market_pairs, 'TimeBar', TIME_BAR_SIZES[base])
    #tasks.extend(gen_tasks(exchange_market_pairs, 'TickBar', TICK_BAR_SIZES[base]))
    #tasks.extend(gen_tasks(exchange_market_pairs, 'VolumeBar', VOLUME_BAR_SIZES[base]))
    #tasks.extend(gen_tasks(exchange_market_pairs, 'DollarBar', DOLLAR_BAR_SIZES[base]))
    #lst = []
    with ProgressBar():
        lst = db.from_sequence(tasks).map(lambda t: segmented_linear_regression(t[0], t[1], t[2])).compute()
        return pd.DataFrame(lst)
    #for t in tqdm(tasks):
        #lst.append(segmented_linear_regression(t[0], t[1], t[2]))

In [21]:
df_btc = batch('BTC')

[########################################] | 100% Completed | 17min 29.0s


In [22]:
df_btc

,exchange,market_type,pair,bar_type,bar_size,R2,MSE,MAE,AIC,BIC
0,Binance,Spot,BTC_USDT,TimeBar,4000,0.999957,6.978550,0.894064,5.078765e+07,5.078766e+07
1,Binance,Spot,BTC_USDT,TimeBar,8000,0.999941,9.529206,1.152789,5.893107e+07,5.893108e+07
2,Binance,Spot,BTC_USDT,TimeBar,10000,0.999936,10.367995,1.259144,6.113637e+07,6.113639e+07
3,Binance,Swap,BTC_USDT,TimeBar,4000,0.999978,3.732232,0.947521,3.314491e+07,3.314493e+07
4,Binance,Swap,BTC_USDT,TimeBar,8000,0.999964,6.176861,1.277974,4.582407e+07,4.582408e+07
5,Binance,Swap,BTC_USDT,TimeBar,10000,0.999953,7.916830,1.420874,5.206999e+07,5.207001e+07
6,BitMEX,Swap,BTC_USD,TimeBar,4000,0.999958,7.306259,0.973957,4.502050e+07,4.502052e+07
7,BitMEX,Swap,BTC_USD,TimeBar,8000,0.999936,11.145304,1.300851,5.458015e+07,5.458016e+07
8,BitMEX,Swap,BTC_USD,TimeBar,10000,0.999926,12.828652,1.435483,5.776444e+07,5.776446e+07
9,Huobi,Spot,BTC_USDT,TimeBar,4000,0.999983,2.778120,0.819746,1.934723e+07,1.934724e+07


In [23]:
df_eth = batch('ETH')

[########################################] | 100% Completed | 13min 53.1s


In [24]:
df_eth

,exchange,market_type,pair,bar_type,bar_size,R2,MSE,MAE,AIC,BIC
0,Binance,Spot,ETH_USDT,TimeBar,4000,0.999993,0.001822,0.019161,-4.247446e+07,-4.247445e+07
1,Binance,Spot,ETH_USDT,TimeBar,8000,0.999990,0.002599,0.025452,-4.008192e+07,-4.008191e+07
2,Binance,Spot,ETH_USDT,TimeBar,10000,0.999989,0.003001,0.028037,-3.911407e+07,-3.911405e+07
3,Binance,Swap,ETH_USDT,TimeBar,4000,0.999995,0.001388,0.019069,-4.988583e+07,-4.988581e+07
4,Binance,Swap,ETH_USDT,TimeBar,8000,0.999992,0.002230,0.025366,-4.629270e+07,-4.629269e+07
5,Binance,Swap,ETH_USDT,TimeBar,10000,0.999990,0.002649,0.028050,-4.498578e+07,-4.498577e+07
6,BitMEX,Swap,ETH_USD,TimeBar,4000,0.999972,0.007973,0.027523,-1.350862e+07,-1.350861e+07
7,BitMEX,Swap,ETH_USD,TimeBar,8000,0.999961,0.010828,0.035204,-1.265275e+07,-1.265274e+07
8,BitMEX,Swap,ETH_USD,TimeBar,10000,0.999952,0.013373,0.038902,-1.206264e+07,-1.206262e+07
9,Huobi,Spot,ETH_USDT,TimeBar,4000,0.999995,0.001616,0.019418,-5.967207e+07,-5.967205e+07


## References

* [Probabilistic Model Selection with AIC, BIC, and MDL](https://machinelearningmastery.com/probabilistic-model-selection-measures/)
* [AIC/BIC for a segmented regression model?](https://stats.stackexchange.com/questions/337852/aic-bic-for-a-segmented-regression-model)
* [Linear Regression With Numpy - Developers Area](https://devarea.com/linear-regression-with-numpy/)
* [What's the relationship between mean squared error and likelihood? - Quora](https://www.quora.com/Whats-the-relationship-between-mean-squared-error-and-likelihood)
* [numpy.linalg.lstsq - NumPy](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)
* [scipy.stats.linregress - SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html)
* [Ordinary Least Squares - statsmodels](https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html)